In [1]:
import os
import numpy as np
import pandas as pd
import glob
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import sys
sys.path.append("..")
from train import BodyPartsMeasurementTrainer
from body_parts_measurement_data_generator import BodyPartsMeasurementDataGenerator
import tensorflow as tf
from evaluator import Evaluator
from model.model import get_model
from tqdm import tqdm
import cv2

2024-10-14 22:30:01.606331: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-14 22:30:01.643814: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-14 22:30:01.644725: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-14 22:30:02.225352: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# path_dataset = "D:\\data\\body_parts_measurement"
path_dataset = "/home/juhwan/Documents/capstone/body_part_measurement/dataset/korean_body/validation"
data_type = "raw"

In [6]:
config = {
    # model
    'input_shape': [256,256,3],     # 훈련 이미지 크기
    'batch_size': 1,                # 배치 사이즈
    'path_pretrained': None,        # pretrained 모델 경로
    'type_backbone': "blazepose",   # backbone type (blazepose, mobilenet_v3)
    
    # loss
    'type_loss_fn': 'wing',         # 손실 함수 설정 (wing, mae)
    
    # data
    'seg_shape': [64,64],           # segmentation 크기 *미사용
    'path_classes': "../seg_classes.txt",   # segmentation class 정보 *미사용
    'shuffle': True,                # 데이터 섞기
    'is_normalized': False,         # normalize 데이터
    'is_with_seg': False,           # segmentation 사용 여부 *미사용
    #'path_dataset': "D:\\data\\body_parts_measurement", # 데이터 경로
    'path_dataset': path_dataset, # 데이터 경로
    ## attention type              
    'type_attention': "regression", # attention 종류 (regression, categorical, none)
    'num_category_bmi': 10,         # categorical 시의 bmi category 갯수 변수
    'num_category_height': 10,      # categorical 시의 height category 갯수 변수

    # train
    'epochs': 30,                   # 훈련 epoch 수
    'eval_term': 1                  # 평가 빈도
}

In [7]:
model = get_model(config)
# model.load_weights('blazepose_attention_31features_731train_mf/blazepose_attention_0_2.994556342230903.h5')
#model.load_weights('blazepose_attention_31features_1002train_mf/blazepose_attention_0_3.2034787193590604.h5')
model.load_weights('blazepose_attention_0_3.2034787193590604.h5')

In [10]:
list_user_dirs = os.listdir(path_dataset + os.path.sep + data_type)
list_user_dirs.sort()
print("list_user_dirs",list_user_dirs)
list_data = []
min_body_parts_measurement = np.full(37, 10000, dtype=np.float32)
max_body_parts_measurement = np.zeros(37, dtype=np.float32)
for user_dir in list_user_dirs :
    path_full_user_dir = path_dataset + os.path.sep + data_type + os.path.sep + user_dir
    path_user_csv = path_full_user_dir + os.path.sep + "csv" + os.path.sep + user_dir + ".csv"
    # df = pd.read_csv(path_user_csv, encoding = "ISO-8859-1")
    df = pd.read_csv(path_user_csv, encoding = "ISO-8859-1")
    body_parts_measurement = list(df.iloc[[1]].values[0][3:-1])
    # # change gender to digit
    # if body_parts_measurement[-2] == "F" : 
    #     body_parts_measurement[-2] = "1"
    # elif body_parts_measurement[-2] == "M" :
    #     body_parts_measurement[-2] = "0"
    # else :
    #     print("wrong in ",user_dir)
    # # get bmi
    # height = float(body_parts_measurement[0])
    # weight = float(body_parts_measurement[32])
    # attention_features = np.expand_dims(np.array([height, weight]),axis=0)
    # bmi = weight / (height/100. * height/100.)
    # body_parts_measurement.append(bmi)
    print("body_parts_measurement",user_dir,len(body_parts_measurement),list(body_parts_measurement))
    # body_parts_measurement = body_parts_measurement[1:32]            
    # body_parts_measurement = np.array(body_parts_measurement, dtype=np.float32)
    # ##### get list of images
    # path_user_image_dir = path_full_user_dir + os.path.sep + "image"
    # # get only images for attention posture
    # list_path_images = glob.glob(path_user_image_dir + os.path.sep + "*.jpg")
    # for path_image in list_path_images :
    #     image = cv2.imread(path_image)
    #     image = cv2.resize(image, (256, 256))
    #     input_image = np.expand_dims(image, axis=0)
    #     pd_parts_measurements = model.predict([input_image, attention_features])
        # batch_diff_parts_measurements = abs(batch_body_parts_measurement - pd_parts_measurements)
        # batch_diff_percentage_parts_measurements = abs(batch_diff_parts_measurements / batch_body_parts_measurement)
    

list_user_dirs ['F001', 'F002', 'F003', 'F004', 'M001', 'M002', 'M003', 'M004']
body_parts_measurement F001 36 ['161.4', '136.1', '80.1', '119.8', '94.2', '73.2', '41.6', '55.5', '35.2', '90.6', '80.5', '83.4', '98,6', '58.4', '38', '38', '21.3', '25', '29.3', '24', '16.7', '30.1', '52.1', '35.4', '21.2', '10.4', '22.9', '9.9', '12.9', '16', '9.2', '7.8', '62.9', '32.8', 'F', '37']
body_parts_measurement F002 36 ['168.1', '140.4', '84.7', '123.3', '107', '77.3', '45.2', '54.7', '31.8', '88.5', '74.4', '81.5', '92', '51.3', '36', '35.5', '20.1', '24.2', '26.5', '22.7', '13.6', '30.8', '54.6', '35.4', '22.1', '10.8', '24.5', '9.3', '13', '17', '9.2', '7', '56.1', '31.5', 'F', '26']
body_parts_measurement F003 36 ['163.4', '138.2', '81.9', '120.6', '97.1', '74.8', '42.3', '52', '30.5', '84.5', '76.5', '79.6', '88.2', '52.8', '32.9', '34.4', '18.9', '23.2', '26.3', '21', '14.2', '30.6', '52.5', '35.4', '19.2', '9.7', '22.4', '9.2', '11.2', '16.9', '9.7', '7.6', '53.2', '25.2', 'F', '39']
b